# IT3212 - Assignment 2

## Setup

In [96]:
import pandas as pd

filepath = '../data/disaster-tweets.csv'
df = pd.read_csv(filepath, encoding='ISO-8859-1')

## 1. Implement the preprocessing

### 1.1 Removing data

In [97]:
# Only keep the columns we need
original_features = [
  '_unit_id', '_golden', '_unit_state', '_trusted_judgments',
  '_last_judgment_at', 'choose_one', 'choose_one:confidence',
  'choose_one_gold', 'keyword', 'location', 'text', 'tweetid', 'userid'
  ]
features_to_keep = ['choose_one', 'choose_one:confidence', 'keyword', 'location', 'text']
df = df[features_to_keep]
print(df.columns)

# Remove rows with a confidence score less than 1.0
confidence_threshold = 1.0
df = df[df['choose_one:confidence'] >= confidence_threshold]

# Write the updated dataframe to a new CSV file
df.to_csv('./results/01-remove-features.csv', index=False)

Index(['choose_one', 'choose_one:confidence', 'keyword', 'location', 'text'], dtype='object')


### 1.2 Text processing

In [98]:
from urllib.parse import unquote

# Clean `keyword` column.
def clean_keyword(keyword: str):
    return unquote(keyword) if pd.notnull(keyword) else ''

df['keyword'] = df['keyword'].apply(clean_keyword)

# Write the updated dataframe to a new CSV file
df.to_csv('./results/02-clean-keywords.csv', index=False)


In [99]:
# Remove links from the `text` column
import string
import re


def clean_text(text: str):
  text = re.sub(r'https?://\S+', '', text)  # Remove link
  text = re.sub(r'\n', ' ', text)  # Remove line breaks
  # Remove leading, trailing, and extra spaces
  text = re.sub('\s+', ' ', text).strip()
  return text


df['cleaned_text'] = df['text'].apply(lambda x: clean_text(x))

# Remove punctuation from the `text` column
df['cleaned_text'] = df['text'].apply(lambda x: x.translate(
    str.maketrans('', '', string.punctuation)))

# Write the updated dataframe to a new CSV file
df.to_csv('./results/03-clean-text.csv', index=False)


In [100]:
# Remove English stopwords
import nltk

nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')

# Clean `text` column.
def clean_text(text: str):
    text = text.lower()
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text


df['cleaned_text'] = df['text'].apply(clean_text)

# Write the updated dataframe to a new CSV file
df.to_csv('./results/04-without-stopwords.csv', index=False)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/magnusrodseth/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [101]:
# Lemmatizing the `text` column
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

# Clean `text` column.
def clean_text(text: str):
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])
    return text


df['cleaned_text'] = df['text'].apply(clean_text)

# Write the updated dataframe to a new CSV file
df.to_csv('./results/05-lemmatized.csv', index=False)


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/magnusrodseth/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [102]:
# Remove special characters
def clean_text(text: str):
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text

df['cleaned_text'] = df['text'].apply(clean_text)

# Write the updated dataframe to a new CSV file
df.to_csv('./results/06-without-special-chars.csv', index=False)

### 1.3 Handling categorical data

In [103]:
# Remove rows with `choose_one` value of `Can't Decide`
df = df[df['choose_one'] != "Can't Decide"]

# Replace the `choose_one` column with a new column called `target`, where `Relevant` is `1` and `Not Relevant` is `0`
df['target'] = df['choose_one'].apply(lambda x: 1 if x == 'Relevant' else 0)

# Write the updated dataframe to a new CSV file
df.to_csv('./results/07-with-target-variable.csv', index=False)

### 1.4 Duplicated data

In [106]:
# Remove duplicated rows with respect to the `text` column
df = df.drop_duplicates(subset=['text'])

print(df.head())

# Write the updated dataframe to a new CSV file
df.to_csv('./results/08-without-duplicates-in-text.csv', index=False)

  choose_one  choose_one:confidence keyword location  \
0   Relevant                    1.0              NaN   
1   Relevant                    1.0              NaN   
2   Relevant                    1.0              NaN   
4   Relevant                    1.0              NaN   
5   Relevant                    1.0              NaN   

                                                text  \
0                 Just happened a terrible car crash   
1  Our Deeds are the Reason of this #earthquake M...   
2  Heard about #earthquake is different cities, s...   
4             Forest fire near La Ronge Sask. Canada   
5  All residents asked to 'shelter in place' are ...   

                                        cleaned_text  target  
0                 Just happened a terrible car crash       1  
1  Our Deeds are the Reason of this earthquake Ma...       1  
2  Heard about earthquake is different cities sta...       1  
4              Forest fire near La Ronge Sask Canada       1  
5  All resi

## 2. Extracting features

> TODO: Extract features from the preprocessed data

## 3. Selecting features

> TODO: Select features from the extracted features

## 4. Implement basic modelling methods